# DP Simulation of topology with slack, line and VSI

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import dpsimpy

# %matplotlib widget

## Simulation

### Parameterization

In [ ]:
final_time = 1.0
time_step = 1e-5
sim_name = "DP_Slack_PiLine_VSI_with_PF_Init"
pv_with_control = True
cmd_scale_P = 1.0
cmd_scale_I = 1.0
V_nom = 20e3

### Powerflow for Initialization

In [ ]:
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# Components
n1_pf = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_pf = dpsimpy.sp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(voltage_set_point=V_nom)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

line_pf = dpsimpy.sp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_pf.set_parameters(R=0.5*5, L=0.5/314*5, C=50e-6/314*5)
line_pf.set_base_voltage(V_nom)

load_pf = dpsimpy.sp.ph1.Load('Load', dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power=-100e3, reactive_power=-50e3, nominal_voltage=20e3)
load_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PQ)

# Topology
extnet_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
load_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [extnet_pf, line_pf, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('v1', 'v', n1_pf)
logger_pf.log_attribute('v2', 'v', n2_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
time_step_dp = time_step
final_time_dp = final_time + time_step_dp
sim_name_dp = sim_name + "_DP"
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dp)

# Components
gnd = dpsimpy.dp.SimNode.gnd
n1_dp = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_dp = dpsimpy.dp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_dp = dpsimpy.dp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_dp.set_parameters(complex(V_nom, 0))

line_dp = dpsimpy.dp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_dp.set_parameters(series_resistance=0.5*5, series_inductance=(0.5/314)*5, parallel_capacitance=(50e-6/314)*5)

pv = dpsimpy.dp.ph1.AvVoltageSourceInverterDQ("pv", "pv", dpsimpy.LogLevel.debug, with_trafo=True)
pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=V_nom, p_ref=100e3, q_ref=50e3)
pv.set_controller_parameters(Kp_pll=cmd_scale_P * 0.25, Ki_pll=cmd_scale_I * 0.2,
                             Kp_power_ctrl=cmd_scale_P *0.001, Ki_power_ctrl=cmd_scale_I * 0.008 ,
                             Kp_curr_ctrl=cmd_scale_P * 0.3, Ki_curr_ctrl=cmd_scale_I * 1, omega_cutoff=2*np.pi*50)
pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
pv.set_transformer_parameters(nom_voltage_end_1=V_nom, nom_voltage_end_2=1500,
                              ratio_abs=V_nom / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
pv.set_initial_state_values(p_init=100e3, q_init=50e3, phi_d_init=0, phi_q_init=0, gamma_d_init=0, gamma_q_init=0)
pv.with_control(pv_with_control)

# Topology
extnet_dp.connect([n1_dp])
line_dp.connect([n1_dp, n2_dp])
pv.connect([n2_dp])
system_dp = dpsimpy.SystemTopology(50, [n1_dp, n2_dp], [extnet_dp, line_dp, pv])

# Initialization of dynamic topology
system_dp.init_with_powerflow(system_pf)

# Logging
logger_dp = dpsimpy.Logger(sim_name_dp)
logger_dp.log_attribute('v1', 'v', n1_dp)
logger_dp.log_attribute('v2', 'v', n2_dp)
logger_dp.log_attribute('i12', 'i_intf', line_dp)


input_names = [
    "pv_powerctrl_input_pref", "pv_powerctrl_input_qref", "pv_powerctrl_input_vcd",
    "pv_powerctrl_input_vcq", "pv_powerctrl_input_ircd", "pv_powerctrl_input_ircq"
]
logger_dp.log_attribute(input_names, 'powerctrl_inputs', pv)

state_names = [
    "pv_powerctrl_state_p", "pv_powerctrl_state_q", "pv_powerctrl_state_phid",
    "pv_powerctrl_state_phiq", "pv_powerctrl_state_gammad", "pv_powerctrl_state_gammaq"
]
logger_dp.log_attribute(state_names, 'powerctrl_states', pv)

output_names = [
    "pv_powerctrl_output_vsd", "pv_powerctrl_output_vsq"
]

logger_dp.log_attribute(output_names, 'powerctrl_outputs', pv)

logger_dp.log_attribute('pv_v_intf', 'v_intf', pv)
logger_dp.log_attribute('pv_i_intf', 'i_intf', pv)
logger_dp.log_attribute('pv_pll_output', 'pll_output', pv)
logger_dp.log_attribute('pv_vsref', 'Vsref', pv)
logger_dp.log_attribute('pv_vs', 'Vs', pv)

# load step sized in absolute terms
load_switch = dpsimpy.dp.ph1.Switch("Load_Add_Switch_n2", dpsimpy.LogLevel.debug)
connection_node = system_dp.node('n2')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 10e6
load_switch.set_parameters(1e9, resistance)
load_switch.open()
system_dp.add(load_switch)
system_dp.connect_component(load_switch, [gnd, system_dp.node('n2')])
logger_dp.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent(np.round(5.0/time_step)*time_step, load_switch, True)

# Simulation
sim_dp = dpsimpy.Simulation(sim_name_dp, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step_dp)
sim_dp.set_final_time(final_time_dp)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.add_event(load_step_event)
sim_dp.add_logger(logger_dp)
sim_dp.run()

## PF results

In [ ]:
modelName = 'DP_Slack_PiLine_VSI_with_PF_Init_PF'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

## DP results

In [ ]:
modelName = 'DP_Slack_PiLine_VSI_with_PF_Init_DP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

In [ ]:
fig = plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name == 'v1' or ts_name == 'v2':
        plt.plot(ts_obj.time, ts_obj.abs().values / 1000, label=ts_name)
#for ts_name, ts_obj  in ts_dpsim_pf.items():
#    if ts_name == 'v1' or ts_name == 'v2':
#        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name+'_pf', linestyle=':')
plt.xlim(4.999, 5.004)
fig.legend(loc='upper center', ncol=2)
plt.xlabel('Zeit [s]')
plt.ylabel('Spannung [kV]')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name in ['i12', 'switchedload_i']:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.xlim(4.999, 5.004)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
ts_5 = ts_dpsim['i12']
ts_5_deriv = (ts_5.abs().values[1:] - ts_5.abs().values[:-1])
plt.plot(ts_5.time[1:], ts_5_deriv, label='Änderung des Stroms i12')
#plt.plot(ts_5.time, ts_5.abs().values)
plt.xlim([4.999, 5.004])
#plt.ylim([-3, 3])

fig.legend(loc='upper center')
plt.xlabel('Zeit [s]')
plt.ylabel('Stromänderung [A/Zeitschritt]')
plt.show()


In [ ]:
#ts = ts_dpsim['i12'].abs()
#
## Calculate end value
#A_end = ts.values[500400]
##A_end = ts.values[10008]
#print("A_end = " + str(A_end))
#
## Calculate damping
#tmax1 = ts.values.argmax()
#tmin1 = ts.values[tmax1:].argmin() + tmax1
#print("tmax1 = " + str(tmax1))
#print("tmin1 = " + str(tmin1))
#A1 = ts.values[tmax1]
#A2 = A_end + (A_end - ts.values[tmin1])
#print("A1 = " + str(A1))
#print("A2 = " + str(A2))
#D = 1 / np.sqrt(1 + (np.pi / np.log(A1 / A2) ** 2)) * 15
#print("D = " + str(D))
#
## Calculate time coefficient
#T0 = 2 * (ts.time[tmin1] - ts.time[tmax1])
#T = T0 * np.sqrt(1 - D) / (2 * np.pi)
#print("T0 = " + str(T0))
#print("T = " + str(T))

In [ ]:
#fig = plt.figure(figsize=(12,6))

#D = 0.57
#T = 0.000208
#lti = sig.lti([A_end], [T**2, 2*D*T, 1])
#t, sys_step_response = sig.step(lti, T=ts.time[500000:500400])
##t, sys_step_response = sig.step(lti, T=ts.time[10000:10008])

#plt.plot(t, ts.abs().values[500000:500400], label='Verlauf i12 aus Simulation')
##plt.plot(t, ts.abs().values[10000:10008], label='Verlauf i12 aus Simulation')
#plt.plot(t, sys_step_response, label='Verlauf des nachgebildeten PT2-Glieds')
#fig.legend(loc='upper center')
#plt.xlabel('Zeit [s]')
#plt.ylabel('Stromstärke [A]')
#pass

In [ ]:
#plt.figure(figsize=(12,6))

#freq = np.arange(0, 2000)
#w, mag = lti.freqresp(w=2*np.pi*freq)

#plt.plot(freq, abs(mag))
#plt.xlabel('Frequenz [Hz]')
#plt.ylabel('Gain')
#pass